***All data in this workbook has been obfuscated and then randomised for privacy & legal reasons***

In [1]:
import gc
import pandas as pd
import numpy as np
from copy import deepcopy
from math import radians, sqrt, sin, cos, atan2

In [2]:
# Define the earth's curvature distance calculation that will be used later in this workbook
def geocalc(lat1, lon1, lat2, lon2):
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon1 - lon2

    EARTH_R = 6372.8

    y = sqrt(
        (cos(lat2) * sin(dlon)) ** 2
        + (cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dlon)) ** 2
        )
    x = sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(dlon)
    c = atan2(y, x)
    return EARTH_R * c


In [3]:
schoolData = pd.read_csv('school_master.csv', encoding='latin-1')

***Prepare the data set for use***

In [4]:
d = deepcopy(schoolData[['ACARA School ID', 'Latitude', 'Longitude', 'ICSEA', 'Teaching Staff']])
print('Dropping %i schools with no ICSEA' % len(d[(d['ICSEA'].isnull())]))
d = d[(d['ICSEA'].notnull())]
print(len(d))
d['Teaching Staff'].fillna(1, inplace=True)
d['joiner'] = 1

print('Outer joining all schools...')
d = pd.merge(d, d, on='joiner', how='outer')
d.drop('joiner', axis=1, inplace=True)
print('Done joining all schools.')

gc.collect()

Dropping 820 schools with no ICSEA
8726
Outer joining all schools...
Done joining all schools.


35

In [5]:
print('Removing self-references...')
d = d[d['ACARA School ID_x'] != d['ACARA School ID_y']]
print('Done removing self-references.')
print('Data set is length %i' % len(d))

gc.collect()

Removing self-references...
Done removing self-references.
Data set is length 76134350


0

In [6]:
d.rename({'distances': 'distance'}, axis=1, inplace=True)

***Calculate the full distance matrix***

In [7]:
lat1s = d['Latitude_x'].values
lon1s = d['Longitude_x'].values
lat2s = d['Latitude_y'].values
lon2s = d['Longitude_y'].values

distances = np.zeros(len(d))

print('Performing geo-calculations...')
for i in range(len(distances)):
    if i % 1000000 == 0:
        print(i, end=', ')
    distances[i] = geocalc(lat1s[i], lon1s[i], lat2s[i], lon2s[i])
print('Done.')
d['distance'] = distances
print('Done performing geo-calculations.')

d.head(3)

Performing geo-calculations...
0, 1000000, 2000000, 3000000, 4000000, 5000000, 6000000, 7000000, 8000000, 9000000, 10000000, 11000000, 12000000, 13000000, 14000000, 15000000, 16000000, 17000000, 18000000, 19000000, 20000000, 21000000, 22000000, 23000000, 24000000, 25000000, 26000000, 27000000, 28000000, 29000000, 30000000, 31000000, 32000000, 33000000, 34000000, 35000000, 36000000, 37000000, 38000000, 39000000, 40000000, 41000000, 42000000, 43000000, 44000000, 45000000, 46000000, 47000000, 48000000, 49000000, 50000000, 51000000, 52000000, 53000000, 54000000, 55000000, 56000000, 57000000, 58000000, 59000000, 60000000, 61000000, 62000000, 63000000, 64000000, 65000000, 66000000, 67000000, 68000000, 69000000, 70000000, 71000000, 72000000, 73000000, 74000000, 75000000, 76000000, Done.
Done performing geo-calculations.


,ACARA School ID_x,Latitude_x,Longitude_x,ICSEA_x,Teaching Staff_x,ACARA School ID_y,Latitude_y,Longitude_y,ICSEA_y,Teaching Staff_y,distance
1,1,-43.317561,147.246052,1063.0,29.0,2,-43.310083,147.013034,1130.0,40.0,18.876279
2,1,-43.317561,147.246052,1063.0,29.0,3,-43.165474,146.943967,969.0,20.0,29.753167
3,1,-43.317561,147.246052,1063.0,29.0,4,-43.164652,147.239683,987.0,28.0,17.015299


In [8]:
# Limit the distance to the highest used, to cut down on calculation size in the next step
x = d[d['distance'] <= 70]

***Create school-school distance pairs for easy display in Tableau***

In [9]:
y = d[d['distance'] <= 1000]
cFrames = []
dist = 70
for dist in [1000, 70, 60, 50]:
    # These are the set distances that the calculations need to be done for
    a = y[y['distance'] <= dist]
    a = a[['ACARA School ID_x', 'ICSEA_x', 'ICSEA_y', 'Teaching Staff_x', 'Teaching Staff_y']]
    a['icsea_rank'] = a.groupby('ACARA School ID_x')['ICSEA_y'].rank(ascending=True, method='first')
    a['teachers_rank'] = a.groupby('ACARA School ID_x')['Teaching Staff_y'].rank(ascending=True, method='first')
    out = pd.DataFrame()
    out['ACARA School ID'] = a['ACARA School ID_x'].unique()
    for i in range(1, 4):
        print('Calculating range ' + str(i+1) + '...')
        b = a[a['icsea_rank'] == i].drop_duplicates(subset=['ACARA School ID_x', 'icsea_rank'])
        # If statements are for the 1000 case, we want them to just be the school's personal numbers so they can't be
        # part of the normal loop
        if dist == 1000:
            b['icsea_minimum_' + str(i+1)] = b['ICSEA_x']
        else:
            b['icsea_minimum_' + str(i+1)] = np.maximum(b['ICSEA_x'], b['ICSEA_y'])
        out = pd.merge(out, deepcopy(b[['ACARA School ID_x', 'icsea_minimum_' + str(i+1)]]), left_on='ACARA School ID', right_on='ACARA School ID_x', how='left')
        b = a[a['teachers_rank'] == i].drop_duplicates(subset=['ACARA School ID_x', 'teachers_rank'])
        if dist == 1000:
            b['teachers_minimum_' + str(i+1)] = b['Teaching Staff_x']
        else:
            b['teachers_minimum_' + str(i+1)] = np.minimum(b['Teaching Staff_x'], b['Teaching Staff_y'])
        out = pd.merge(out, deepcopy(b[['ACARA School ID_x', 'teachers_minimum_' + str(i+1)]]), left_on='ACARA School ID', right_on='ACARA School ID_x', how='left')
    out['distance'] = dist
    out = out[['ACARA School ID', 'distance', 'icsea_minimum_2', 'teachers_minimum_2', 'icsea_minimum_3', 'teachers_minimum_3', 'icsea_minimum_4', 'teachers_minimum_4']]
    cFrames.append(deepcopy(out))
out = pd.concat(cFrames)
out.head(5)

Calculating range 2...
Calculating range 3...
Calculating range 4...
Calculating range 2...
Calculating range 3...
Calculating range 4...
Calculating range 2...
Calculating range 3...
Calculating range 4...
Calculating range 2...
Calculating range 3...
Calculating range 4...


,ACARA School ID,distance,icsea_minimum_2,teachers_minimum_2,icsea_minimum_3,teachers_minimum_3,icsea_minimum_4,teachers_minimum_4
0,1,1000,1063.0,29.0,1063.0,29.0,1063.0,29.0
1,2,1000,1130.0,40.0,1130.0,40.0,1130.0,40.0
2,3,1000,969.0,20.0,969.0,20.0,969.0,20.0
3,4,1000,987.0,28.0,987.0,28.0,987.0,28.0
4,5,1000,1006.0,14.0,1006.0,14.0,1006.0,14.0


***Export***

In [10]:
schoolData = pd.merge(out, schoolData, on='ACARA School ID', how='right')
schoolData = schoolData[schoolData['distance'].notnull()]

In [11]:
schoolData.to_csv('School Geo Export.csv', index=False)
schoolData.head(3)

,ACARA School ID,distance,icsea_minimum_2,teachers_minimum_2,icsea_minimum_3,teachers_minimum_3,icsea_minimum_4,teachers_minimum_4,AGE ID,School Name,...,C2018 placements,C2018,Total placements,Current TFA partner school,TFA placement school,2018 PAN member,TLA,TTL school,Alumni at school,Duplicates
0,1,1000.0,1063.0,29.0,1063.0,29.0,1063.0,29.0,3.0,obfuscated_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,70.0,1063.0,3.0,1063.0,5.0,1063.0,7.0,3.0,obfuscated_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,60.0,1063.0,3.0,1063.0,7.0,1063.0,8.0,3.0,obfuscated_1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
